# Soccer Match Outcome Predictor

The following code will build a soccer match outcome prediction model using nural networks with team embeddings and match history embeddings.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Concatenate, Dropout, Flatten, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from keras import layers, models, optimizers, callbacks
import keras_tuner as kt

## Preprocessing the Dataset


Loading the Dataset

In [ ]:
df = pd.read_csv('matches_expanded.csv')
df.head(10)

Cleaning the Dataframe

In [ ]:
df.columns = df.columns.str.lower()

df['home_ball_possession'] = df['home_ball_possession'].str.replace('%', '', regex=True).astype(float)
df['home_pass_accuracy'] = df['home_pass_accuracy'].str.replace('%', '', regex=True).astype(float)
df['away_ball_possession'] = df['away_ball_possession'].str.replace('%', '', regex=True).astype(float)
df['away_pass_accuracy'] = df['away_pass_accuracy'].str.replace('%', '', regex=True).astype(float)


df.head(10)

Converting Team Names to Integer IDs

In [ ]:
team_encoder = LabelEncoder()
df['home_team_id'] = team_encoder.fit_transform(df['home_team'])
df['away_team_id'] = team_encoder.transform(df['away_team'])
df.head(10)

Generating Historical Performace Features



In [ ]:
N = 5  # Number of past matches to consider

def get_past_stats(df, team_col, stat_col, N):
    """
    Computes rolling average for the last N matches per team.
    If rolling value is NaN (not enough history), fill it with the current match's stat.
    """
    rolling_avg = df.groupby(team_col)[stat_col].transform(lambda x: x.shift(1).rolling(N, min_periods=1).mean())
    
    # Fill NaNs with the current match stat as a fallback
    rolling_avg = rolling_avg.fillna(df[stat_col])
    
    return rolling_avg

historical_features = ["goals", "ball_possession", "pass_accuracy", "total_shots", "expected_goals"]

for feature in historical_features:
    home_feature_col = f'home_{feature}'
    away_feature_col = f'away_{feature}'

    df[f'{home_feature_col}_hist'] = get_past_stats(df, 'home_team', home_feature_col, N=5)
    df[f'{away_feature_col}_hist'] = get_past_stats(df, 'away_team', away_feature_col, N=5)


df.head(10)
df.to_csv('cleaned_matches_expanded.csv', index=False)

Standard Scaling

In [ ]:
numerical_cols = [col for col in df.columns if '_hist' in col or col in [
    'home_ball_possession', 'home_pass_accuracy', 'home_total_shots', 'home_expected_goals'
    'away_ball_possession', 'away_pass_accuracy', 'away_total_shots', 'away_expected_goals'
]]

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

Label Encode Target (Win/Draw/Loss)

In [ ]:
# Assuming perspective is home team
# Win = 2, Draw = 0, Loss = 1

outcome_encoder = LabelEncoder()
df['outcome_encoded'] = outcome_encoder.fit_transform(df['match_outcome'])
y = to_categorical(df['outcome_encoded'], num_classes=3)

print(df.columns)
df.head(10)

## Building the Neural Network Model


Spliting Data into Training and Test Datasets

In [ ]:
X = df[numerical_cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

Building the Model

In [ ]:
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(3, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


Training the Model

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

## Evaluating the Model


Predict and Evalutate

In [ ]:
# Predict probabilities on the test set
y_test_probs = model.predict(X_test)

# Predicted class labels (Win, Draw, Loss)
y_test_pred = np.argmax(y_test_probs, axis=1)

# True class labels (converted back from one-hot encoding)
y_test_true = np.argmax(y_test, axis=1)

In [ ]:
print("Accuracy:", accuracy_score(y_test_true, y_test_pred))
print("Classification Report:\n", classification_report(y_test_true, y_test_pred, target_names=outcome_encoder.classes_))


In [ ]:
y_test_pred_labels = outcome_encoder.inverse_transform(y_test_pred)
y_test_true_labels = outcome_encoder.inverse_transform(y_test_true)
# Build a dataframe to compare actual vs predicted
test_results_df = pd.DataFrame({
    'Actual Outcome': y_test_true_labels,
    'Predicted Outcome': y_test_pred_labels,
    'Confidence': np.max(y_test_probs, axis=1),
    'Win Prob': y_test_probs[:, outcome_encoder.transform(['Win'])[0]],
    'Draw Prob': y_test_probs[:, outcome_encoder.transform(['Draw'])[0]],
    'Loss Prob': y_test_probs[:, outcome_encoder.transform(['Loss'])[0]]
})

# Add 'Correct' column (True if prediction matches actual)
test_results_df['Correct'] = (test_results_df['Actual Outcome'] == test_results_df['Predicted Outcome'])

# Show the first few predictions
print(test_results_df.head())
test_results_df.to_csv('test_results_1.csv', index=False)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create confusion matrix
cm = confusion_matrix(y_test_true_labels, y_test_pred_labels, labels=outcome_encoder.classes_)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=outcome_encoder.classes_,
            yticklabels=outcome_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix on Test Set')
plt.show()

In [ ]:
# Group by Actual Outcome and calculate mean accuracy
outcome_accuracy = test_results_df.groupby('Actual Outcome')['Correct'].mean()

# Plot
outcome_accuracy.plot(kind='bar', color='skyblue')
plt.ylabel('Accuracy')
plt.title('Prediction Accuracy per Outcome on Test Set')
plt.ylim(0, 1)
plt.grid(axis='y')
plt.show()


In [ ]:
# Define low confidence threshold
low_confidence_threshold = 0.6

# Create a column marking low-confidence wrong predictions
test_results_df['Low-Confidence Wrong'] = (
    (~test_results_df['Correct']) & (test_results_df['Confidence'] < low_confidence_threshold)
)

# Display only low-confidence wrong predictions
low_conf_wrong_preds = test_results_df[test_results_df['Low-Confidence Wrong']]

print(f"Low-Confidence Wrong Predictions: {len(low_conf_wrong_preds)} matches")
print(low_conf_wrong_preds[['Actual Outcome', 'Predicted Outcome', 'Confidence']])


In [ ]:
# Plot the distribution of confidence scores
plt.figure(figsize=(8,5))
plt.hist(test_results_df['Confidence'], bins=20, color='mediumseagreen', edgecolor='black')
plt.xlabel('Prediction Confidence')
plt.ylabel('Number of Matches')
plt.title('Distribution of Prediction Confidence (Test Set)')
plt.grid(axis='y')
plt.show()

## Hyperparameter Tuning

Defining a Hypermodel

In [ ]:
from keras import layers, models, optimizers
import keras_tuner as kt

def build_model(hp):
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(X_train.shape[1],)))
    
    # Tune number of hidden layers (1–3)
    for i in range(hp.Int('num_layers', 1, 3)):
        # Tune units per layer
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32)
        model.add(layers.Dense(units, activation='relu'))
        
        # Tune dropout rate
        dropout_rate = hp.Float(f'dropout_rate_{i}', min_value=0.1, max_value=0.5, step=0.1)
        model.add(layers.Dropout(dropout_rate))
    
    # Output layer
    model.add(layers.Dense(3, activation='softmax'))  # 3 classes (Win, Draw, Loss)
    
    # Tune learning rate
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer = optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


Hyperparameter Search

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # How many different models to try
    executions_per_trial=1,
    overwrite=True,
    directory='tuning_dir',
    project_name='soccer_match_outcome'
)

# Perform the search
tuner.search(
    X_train, y_train,
    epochs=30,
    validation_split=0.2,
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Validation Accuracy Across Trials

In [ ]:
# Get all the tuning results
tuner_results = tuner.oracle.get_best_trials(num_trials=20)

# Extract val_accuracy from each trial
val_accuracies = [trial.metrics.get_last_value('val_accuracy') for trial in tuner_results]

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(range(1, len(val_accuracies)+1), val_accuracies, marker='o', linestyle='-')
plt.xlabel('Trial')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy across Tuning Trials')
plt.grid(True)
plt.show()


There was no better model found using random search. We are going to use Bayesian optimization as well as smarter search ranges, better validation, and early stopping to hopefully find a better model

Defining a new hypermodel

In [ ]:
def build_model(hp):
    model = models.Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(X_train.shape[1] + 16,)))  # Adjust if you have team embeddings added

    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(
            layers.Dense(
                units=hp.Int(f'units_{i}', min_value=64, max_value=256, step=32),
                activation='relu'
            )
        )
        model.add(
            layers.Dropout(
                rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)
            )
        )

    # Output layer
    model.add(layers.Dense(3, activation='softmax'))

    # Optimizer
    optimizer = optimizers.Adam(
        learning_rate=hp.Float('learning_rate', 1e-4, 5e-3, sampling='log')
    )

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model